# LLM-rankers

This jupyter notebook uses [llm-rankers](https://github.com/ielab/llm-rankers) to re-rank documents in the context [ReNeuIR 2024](https://reneuir.org/). Please look at the [corresponding publications](https://github.com/ielab/llm-rankers#references) for more details.

### Step 1: Import dependencies

In [10]:
import torch
from tqdm import tqdm
import pandas as pd
from tira.third_party_integrations import ir_datasets, persist_and_normalize_run
from llmrankers.setwise import SetwiseLlmRanker
from llmrankers.rankers import SearchResult
import os


### Step 2: Load and Transform the Dataset

In [2]:
# the "from tira.third_party_integrations import ir_datasets" import patches "ir_datasets.load"
# so that it loads the dataset injected into the tira sandbox when executed within the sandbox.
# I.e., we only ensure that it runs on a minimal spot-check dataset here.
dataset = ir_datasets.load('reneuir-2024/re-rank-spot-check-20240624-training')

In [4]:
qid_to_rerank_data = {}

for i in dataset.scoreddocs_iter():
    if i.query_id not in qid_to_rerank_data:
        qid_to_rerank_data[i.query_id] = {'query': i.query.default_text(), 'search_results': []}
    
    qid_to_rerank_data[i.query_id]['search_results']  += [
        SearchResult(docid=i.doc_id, text=i.document.default_text(), score=i.score)
    ]


### Step 3: Re-Rank

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Will use device:', device)

# We make the model injectable into the notebook so that we can run multiple models with the same code.
model = os.environ.get('MODEL', 'google/flan-t5-small')
print('Model is:', model)

ranker = SetwiseLlmRanker(model_name_or_path=model, tokenizer_name_or_path=model, device=device)

print('Ranker is', ranker)

Will use device: cuda
Model is: google/flan-t5-small


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ranker is <llmrankers.setwise.SetwiseLlmRanker object at 0x70ed62807130>


In [7]:
run = []

for qid in tqdm(qid_to_rerank_data):
    query = qid_to_rerank_data[qid]['query']
    search_results = qid_to_rerank_data[qid]['search_results']

    for i in ranker.rerank(query, search_results):
        run += [{"qid": qid, "score": i.score, "docno": i.docid}]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.41s/it]


### Step 4: Persist run file

In [11]:
run = pd.DataFrame(run)

persist_and_normalize_run(run, system_name=f'llm-rankers-{model}', default_output='.')

The run file is normalized outside the TIRA sandbox, I will store it at ".".
Done. run file is stored under "./run.txt".
